<a href="https://www.kaggle.com/code/muheedahmedkhan/spider-crawler-yahoofinance-2pages?scriptVersionId=196531212" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install scrapy

In [2]:
from scrapy import Selector
import requests

/opt/conda/lib/python3.10/site-packages/scrapy/utils/_compression.py:15: ScrapyDeprecationWarning: You have brotlipy installed, and Scrapy will use it, but Scrapy support for brotlipy is deprecated and will stop working in a future version of Scrapy. brotlipy itself is deprecated, it has been superseded by brotlicffi (not currently supported by Scrapy). Please, uninstall brotlipy and install brotli instead. brotlipy has the same import name as brotli, so keeping both installed is strongly discouraged.
  warn(


Taking one of the links of Company profile on the yahoo finance page as an example to check the correctness of Locator

In [3]:
url = 'https://finance.yahoo.com/quote/TSLA/'
html = requests.get(url).content
sel = Selector(text = html)

In [4]:
stock_price = sel.css('fin-streamer[data-field="regularMarketPrice"] > span::text').extract_first()
stock_price

'229.39'

In [5]:
stock_price_change = sel.css('fin-streamer[data-field="regularMarketChange"] > span::text').extract_first()
stock_price_change

'-0.42'

In [6]:
stock_price_change_in_perc = sel.css('fin-streamer[data-field="regularMarketChangePercent"] > span::text').extract_first().replace('(','').replace(')','')
stock_price_change_in_perc

'-0.18%'

In [7]:
beta_5y_monthly = sel.css('li:nth-of-type(10) > span:nth-of-type(2)::text').extract()
beta_5y_monthly

['2.29']

In [8]:
eps_ttm = sel.css('fin-streamer[data-field="trailingPE"]::text')[1].extract()
eps_ttm

'3.56 '

checked all the locators are working fine

In [9]:
import scrapy
from scrapy.crawler import CrawlerProcess

# Create the Spider class
class StockSpider(scrapy.Spider):
    name = "stock_spider"
    start_url = 'https://finance.yahoo.com'
    def start_requests(self):
        url = self.start_url
        yield scrapy.Request(url=url, callback=self.parse)
#         print("Start Request")
#         print(url)

    # First parsing method: extract links to company stock pages
    def parse(self, response):
        stock_links = response.css('a.none-link.fin-size-small.tw-block::attr(href)').extract()
        stock_names = response.css('span.symbol.valid::text').extract()
        print("parse")
        print(stock_links)
        print(stock_names)
        for link, name in zip(stock_links, stock_names):
            link = self.start_url + link 
            yield response.follow(url=link, callback=self.parse_stock_details, meta={'stock_name': name, 'link': response.urljoin(link)})

    # Second parsing method: extract details from each stock page
    def parse_stock_details(self, response):
        stock_name = response.meta['stock_name']
        stock_link = response.meta['link']
        ##########################################################################
        stock_price = response.css('fin-streamer[data-field="regularMarketPrice"] > span::text').extract_first()
        if stock_price :
            stock_price = stock_price.strip()
        else :
            stock_price = response.css('fin-streamer[data-field="regularMarketPrice"]::text').extract_first()
            if stock_price :
                stock_price = stock_price.strip()
        print("Link : ", stock_link, " Field stock_price : ",stock_price )
        ##########################################################################
        change_in_price = response.css('fin-streamer[data-field="regularMarketChange"] > span::text').extract_first()
        if change_in_price:
            change_in_price = change_in_price.strip()
        else :
            change_in_price = response.css('fin-streamer[data-field="regularMarketChange"]::text').extract_first()
            if change_in_price:
                change_in_price = change_in_price.strip()
        print("Link : ", stock_link, " Field change_in_price : ",change_in_price )
        ##########################################################################
        change_in_percentage = response.css('fin-streamer[data-field="regularMarketChangePercent"] > span::text').extract_first()
        if change_in_percentage:
            change_in_percentage=change_in_percentage.strip().replace('(','').replace(')','')
        else :
            change_in_percentage = response.css('fin-streamer[data-field="regularMarketChangePercent"]::text').extract_first()
            if change_in_percentage:
                change_in_percentage=change_in_percentage.strip().replace('(','').replace(')','')
        print("Link : ", stock_link, " Field change_in_percentage : ", change_in_percentage)
        ##########################################################################
        previous_close = response.css('fin-streamer[data-field="regularMarketPreviousClose"] > span::text').extract_first()
        if previous_close:
            previous_close = previous_close.strip()
        else :
            previous_close = response.css('fin-streamer[data-field="regularMarketPreviousClose"]::text').extract_first()
            if previous_close:
                previous_close = previous_close.strip()
        print("Link : ", stock_link, " Field previous_close : ", previous_close)
        ##########################################################################
        days_range = response.css('fin-streamer[data-field="regularMarketDayRange"] > span::text').extract_first()
        if days_range:
            days_range = days_range.strip()
        else :
            days_range = response.css('fin-streamer[data-field="regularMarketDayRange"]::text').extract_first()
            if days_range:
                days_range = days_range.strip()
        print("Link : ", stock_link, " Field days_range : ",days_range )
        ##########################################################################
        market_open_price = response.css('fin-streamer[data-field="regularMarketOpen"] > span::text').extract_first()
        if market_open_price: 
            market_open_price = market_open_price.strip()
        else :
            market_open_price = response.css('fin-streamer[data-field="regularMarketOpen"]::text').extract_first()
            if market_open_price: 
                market_open_price = market_open_price.strip()
        print("Link : ", stock_link, " Field market_open_price : ", market_open_price)
        ##########################################################################
        fifty_two_week_range = response.css('fin-streamer[data-field="fiftyTwoWeekRange"] > span::text').extract_first()
        if fifty_two_week_range:
            fifty_two_week_range = fifty_two_week_range.strip()
        else:
            fifty_two_week_range = response.css('fin-streamer[data-field="fiftyTwoWeekRange"]::text').extract_first()
            if fifty_two_week_range:
                fifty_two_week_range = fifty_two_week_range.strip()
        print("Link : ", stock_link, " Field fifty_two_week_range: ", fifty_two_week_range)
        ##########################################################################
        volume = response.css('fin-streamer[data-field="regularMarketVolume"] > span::text').extract_first()
        if volume:
            volume = volume.strip()
        else:
            volume = response.css('fin-streamer[data-field="regularMarketVolume"]::text').extract_first()
            if volume:
                volume = volume.strip()
        print("Link : ", stock_link, " Field volume: ", volume)
        ##########################################################################
        avg_volume = response.css('fin-streamer[data-field="averageVolume"] > span::text').extract_first()
        if avg_volume:
            avg_volume = avg_volume.strip()
        else:
            avg_volume = response.css('fin-streamer[data-field="averageVolume"]::text').extract_first()
            if avg_volume:
                avg_volume = avg_volume.strip()
        print("Link : ", stock_link, " Field avg_volume: ",avg_volume )
        ##########################################################################
        market_cap = response.css('fin-streamer[data-field="marketCap"] > span::text').extract_first()
        if market_cap:
            market_cap = market_cap.strip()
        else:
            market_cap = response.css('fin-streamer[data-field="marketCap"]::text').extract_first()
            if market_cap:
                market_cap = market_cap.strip()
        print("Link : ", stock_link, " Field market_cap: ",market_cap )
        ##########################################################################
        beta_5y_monthly = response.css('li:nth-of-type(10) > span:nth-of-type(2)::text').extract_first()
        if beta_5y_monthly:
            beta_5y_monthly = beta_5y_monthly.strip()
        print("Link : ", stock_link, " Field beta_5y_monthly: ",beta_5y_monthly )
        ##########################################################################
        pe_ratio_ttm = response.css('fin-streamer[data-field="trailingPE"] > span::text').extract_first()
        if pe_ratio_ttm:
            pe_ratio_ttm = pe_ratio_ttm.strip()
        else:
            pe_ratio_ttm = response.css('fin-streamer[data-field="trailingPE"]::text').extract_first()
            if pe_ratio_ttm:
                pe_ratio_ttm = pe_ratio_ttm.strip()
        print("Link : ", stock_link, " Field pe_ratio_ttm: ",pe_ratio_ttm )
        ##########################################################################
        eps_ttm_elements = response.css('fin-streamer[data-field="trailingPE"] > span::text').extract()
        if len(eps_ttm_elements) > 1:
            eps_ttm = eps_ttm_elements[1].strip()
        else:
            eps_ttm_elements = response.css('fin-streamer[data-field="trailingPE"]::text').extract()
            if len(eps_ttm_elements) > 1:
                eps_ttm = eps_ttm_elements[1].strip()
        print("Link: ", stock_link, " Field eps_ttm: ", eps_ttm)
        ##########################################################################
        earnings_date = response.css('li:nth-of-type(13) > span:nth-of-type(2)::text').extract_first()
        if earnings_date:
            earnings_date = earnings_date.strip()
        print("Link : ", stock_link, " Field earnings_date: ",earnings_date )
        ##########################################################################
        fwd_dividend_and_yield = response.css('li:nth-of-type(14) > span:nth-of-type(2)::text').extract_first()
        if fwd_dividend_and_yield:
            fwd_dividend_and_yield = fwd_dividend_and_yield.strip()
        print("Link : ", stock_link, " Field fwd_dividend_and_yield: ",fwd_dividend_and_yield )
        ##########################################################################
        ex_dividend_date = response.css('li:nth-of-type(15) > span:nth-of-type(2)::text').extract_first()
        if ex_dividend_date:
            ex_dividend_date = ex_dividend_date.strip()
        print("Link : ", stock_link, " Field ex_dividend_date: ",ex_dividend_date )        
        ##########################################################################
        one_year_target_estimate = response.css('fin-streamer[data-field="targetMeanPrice"] > span::text').extract_first()
        if one_year_target_estimate:
            one_year_target_estimate = one_year_target_estimate.strip()
        else:
            one_year_target_estimate = response.css('fin-streamer[data-field="targetMeanPrice"]::text').extract_first()
            if one_year_target_estimate:
                one_year_target_estimate = one_year_target_estimate.strip()
        print("Link : ", stock_link, " Field one_year_target_estimate: ",one_year_target_estimate )
        
        # Collect the data for both positive and negative changes
        stock_data = (stock_name, stock_link, stock_price,change_in_price, change_in_percentage,previous_close,
                     days_range,market_open_price,fifty_two_week_range,volume,avg_volume , market_cap,beta_5y_monthly,pe_ratio_ttm,
                     eps_ttm,earnings_date, fwd_dividend_and_yield, ex_dividend_date, one_year_target_estimate )
        stocks_data.append(stock_data)

stocks_data = []

# Run the Spider
process = CrawlerProcess()
process.crawl(StockSpider)
process.start()

# Print a preview of the stocks data
print("Stocks:")
print(stocks_data)


2024-09-13 15:52:13 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-09-13 15:52:13 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0], pyOpenSSL 24.0.0 (OpenSSL 3.3.1 4 Jun 2024), cryptography 42.0.8, Platform Linux-5.15.154+-x86_64-with-glibc2.31
2024-09-13 15:52:13 [scrapy.addons] INFO: Enabled addons:
[]
2024-09-13 15:52:13 [py.warnings] WARNING: /opt/conda/lib/python3.10/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation 

parse
['/quote/%5EGSPC/', '/quote/%5EDJI/', '/quote/%5EIXIC/', '/quote/%5ERUT/', '/quote/CL%3DF/', '/quote/GC%3DF/', '/quote/SI%3DF/', '/quote/EURUSD%3DX/', '/quote/%5ETNX/', '/quote/GBPUSD%3DX/', '/quote/JPY%3DX/', '/quote/BTC-USD/', '/quote/XRP-USD/', '/quote/%5EFTSE/', '/quote/%5EN225/', '/quote/%5EGSPC/', '/quote/%5EDJI/', '/quote/%5EIXIC/', '/quote/%5ERUT/', '/quote/CL%3DF/', '/quote/GC%3DF/', '/quote/RH/', '/quote/SMMT/', '/quote/ASTS/', '/quote/BNTX/', '/quote/LAUR/', '/quote/ADBE/', '/quote/GRMN/', '/quote/MRNA/', '/quote/BILI/', '/quote/SVNDY/', '/quote/NVDA/', '/quote/TSLA/', '/quote/NIO/', '/quote/PLTR/', '/quote/WBD/', '/quote/ADBE/', '/quote/BA/', '/quote/RH/', '/quote/UBER/', '/quote/ORCL/']
['S&P 500', 'Dow 30', 'Nasdaq', 'Russell 2000', 'Crude Oil', 'Gold', 'Silver', 'EUR/USD', '10-Yr Bond', 'GBP/USD', 'USD/JPY', 'Bitcoin USD', 'XRP USD', 'FTSE 100', 'Nikkei 225', 'S&P 500', 'Dow 30', 'Nasdaq', 'Russell 2000', 'Crude Oil', 'Gold', 'RH', 'SMMT', 'ASTS', 'BNTX', 'LAUR', '

2024-09-13 15:52:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://finance.yahoo.com/quote/%5EGSPC/> (referer: https://finance.yahoo.com)
2024-09-13 15:52:15 [scrapy.core.scraper] ERROR: Spider error processing <GET https://finance.yahoo.com/quote/%5EGSPC/> (referer: https://finance.yahoo.com)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/twisted/internet/defer.py", line 1074, in _runCallbacks
    current.result = callback(  # type: ignore[misc]
  File "/tmp/ipykernel_1008/401069766.py", line 141, in parse_stock_details
    print("Link: ", stock_link, " Field eps_ttm: ", eps_ttm)
UnboundLocalError: local variable 'eps_ttm' referenced before assignment


Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field stock_price :  5,632.34
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field change_in_price :  +36.58
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field change_in_percentage :  +0.65%
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field previous_close :  5,595.76
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field days_range :  5,601.65 - 5,633.86
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field market_open_price :  5,603.34
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field fifty_two_week_range:  4,103.78 - 5,669.67
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field volume:  826,483,686
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field avg_volume:  3,771,102,096
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field market_cap:  None
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field beta_5y_monthly:  None
Link :  https://finance.yahoo.com/quote/%5EGSPC/  Field pe_ratio_ttm:  None


2024-09-13 15:52:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://finance.yahoo.com/quote/%5EFTSE/> (referer: https://finance.yahoo.com)
2024-09-13 15:52:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://finance.yahoo.com/quote/%5EN225/> (referer: https://finance.yahoo.com)
2024-09-13 15:52:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://finance.yahoo.com/quote/SMMT/> (referer: https://finance.yahoo.com)
2024-09-13 15:52:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://finance.yahoo.com/quote/XRP-USD/> (referer: https://finance.yahoo.com)
2024-09-13 15:52:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://finance.yahoo.com/quote/RH/> (referer: https://finance.yahoo.com)
2024-09-13 15:52:16 [scrapy.core.scraper] ERROR: Spider error processing <GET https://finance.yahoo.com/quote/%5EFTSE/> (referer: https://finance.yahoo.com)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/twisted/internet/defer.py", line 1074,

...

Link :  https://finance.yahoo.com/quote/GRMN/  Field stock_price :  170.92
Link :  https://finance.yahoo.com/quote/GRMN/  Field change_in_price :  -11.09
Link :  https://finance.yahoo.com/quote/GRMN/  Field change_in_percentage :  -6.09%
Link :  https://finance.yahoo.com/quote/GRMN/  Field previous_close :  182.01
Link :  https://finance.yahoo.com/quote/GRMN/  Field days_range :  168.18 - 174.65
Link :  https://finance.yahoo.com/quote/GRMN/  Field market_open_price :  173.22
Link :  https://finance.yahoo.com/quote/GRMN/  Field fifty_two_week_range:  99.61 - 184.42
Link :  https://finance.yahoo.com/quote/GRMN/  Field volume:  845,019
Link :  https://finance.yahoo.com/quote/GRMN/  Field avg_volume:  737,874
Link :  https://finance.yahoo.com/quote/GRMN/  Field market_cap:  32.853B
Link :  https://finance.yahoo.com/quote/GRMN/  Field beta_5y_monthly:  1.01
Link :  https://finance.yahoo.com/quote/GRMN/  Field pe_ratio_ttm:  23.90
Link:  https://finance.yahoo.com/quote/GRMN/  Field eps_ttm: 

2024-09-13 15:52:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://finance.yahoo.com/quote/%5ERUT/> (referer: https://finance.yahoo.com)
2024-09-13 15:52:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://finance.yahoo.com/quote/%5EIXIC/> (referer: https://finance.yahoo.com)
2024-09-13 15:52:21 [scrapy.core.scraper] ERROR: Spider error processing <GET https://finance.yahoo.com/quote/%5ERUT/> (referer: https://finance.yahoo.com)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/twisted/internet/defer.py", line 1074, in _runCallbacks
    current.result = callback(  # type: ignore[misc]
  File "/tmp/ipykernel_1008/401069766.py", line 141, in parse_stock_details
    print("Link: ", stock_link, " Field eps_ttm: ", eps_ttm)
UnboundLocalError: local variable 'eps_ttm' referenced before assignment


Link :  https://finance.yahoo.com/quote/ADBE/  Field stock_price :  531.77
Link :  https://finance.yahoo.com/quote/ADBE/  Field change_in_price :  -54.78
Link :  https://finance.yahoo.com/quote/ADBE/  Field change_in_percentage :  -9.34%
Link :  https://finance.yahoo.com/quote/ADBE/  Field previous_close :  586.55
Link :  https://finance.yahoo.com/quote/ADBE/  Field days_range :  526.60 - 537.32
Link :  https://finance.yahoo.com/quote/ADBE/  Field market_open_price :  532.54
Link :  https://finance.yahoo.com/quote/ADBE/  Field fifty_two_week_range:  433.97 - 638.25
Link :  https://finance.yahoo.com/quote/ADBE/  Field volume:  6,170,522
Link :  https://finance.yahoo.com/quote/ADBE/  Field avg_volume:  2,750,303
Link :  https://finance.yahoo.com/quote/ADBE/  Field market_cap:  235.785B
Link :  https://finance.yahoo.com/quote/ADBE/  Field beta_5y_monthly:  --
Link :  https://finance.yahoo.com/quote/ADBE/  Field pe_ratio_ttm:  45.06
Link:  https://finance.yahoo.com/quote/ADBE/  Field eps_t

2024-09-13 15:52:21 [scrapy.core.scraper] ERROR: Spider error processing <GET https://finance.yahoo.com/quote/%5EIXIC/> (referer: https://finance.yahoo.com)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/twisted/internet/defer.py", line 1074, in _runCallbacks
    current.result = callback(  # type: ignore[misc]
  File "/tmp/ipykernel_1008/401069766.py", line 141, in parse_stock_details
    print("Link: ", stock_link, " Field eps_ttm: ", eps_ttm)
UnboundLocalError: local variable 'eps_ttm' referenced before assignment
2024-09-13 15:52:21 [scrapy.core.engine] INFO: Closing spider (finished)
2024-09-13 15:52:21 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 19441,
 'downloader/request_count': 34,
 'downloader/request_method_count/GET': 34,
 'downloader/response_bytes': 11276045,
 'downloader/response_count': 34,
 'downloader/response_status_count/200': 33,
 'downloader/response_status_count/404': 1,
 'dupefilter/filter

Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field stock_price :  17,684.65
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field change_in_price :  +114.97
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field change_in_percentage :  +0.65%
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field previous_close :  17,569.68
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field days_range :  17,564.61 - 17,689.74
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field market_open_price :  17,575.30
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field fifty_two_week_range:  12,543.86 - 18,671.07
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field volume:  1,967,902,000
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field avg_volume:  5,555,922,419
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field market_cap:  None
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field beta_5y_monthly:  None
Link :  https://finance.yahoo.com/quote/%5EIXIC/  Field pe_ratio_ttm: 

In [10]:
# stock_data = (stock_name, stock_link, stock_price,change_in_price, change_in_percentage,previous_close,days_range,
#                      days_range,market_open_price,fifty_two_week_range,volume,avg_volume , market_cap,beta_5y_monthly,pe_ratio_ttm,
#                      eps_ttm,earnings_date, fwd_dividend_and_yield, ex_dividend_date, one_year_target_estimate )
import pandas as pd

df = pd.DataFrame(stocks_data, columns=[
    'Stock Name', 'Stock Link', 'Stock Price', 'Change in Price', 'Change in Percentage', 
    'Previous Close', 'Day\'s Range', 'Market Open Price', '52-Week Range', 
    'Volume', 'Avg Volume', 'Market Cap', 'Beta (5Y Monthly)', 'PE Ratio (TTM)', 
    'EPS (TTM)', 'Earnings Date', 'Forward Dividend & Yield', 'Ex-Dividend Date', 
    '1-Year Target Estimate'
])
df

2024-09-13 15:52:21 [numexpr.utils] INFO: NumExpr defaulting to 4 threads.


,Stock Name,Stock Link,Stock Price,Change in Price,Change in Percentage,Previous Close,Day's Range,Market Open Price,52-Week Range,Volume,Avg Volume,Market Cap,Beta (5Y Monthly),PE Ratio (TTM),EPS (TTM),Earnings Date,Forward Dividend & Yield,Ex-Dividend Date,1-Year Target Estimate
0,SMMT,https://finance.yahoo.com/quote/SMMT/,32.11,+4.70,+17.15%,27.41,27.41 - 32.89,27.50,1.64 - 32.89,"14,368,295","3,623,496",23.597B,-1.00,--,-0.11,"Nov 5, 2024 - Nov 11, 2024",--,--,19.75
1,RH,https://finance.yahoo.com/quote/RH/,320.62,+64.13,+25.00%,256.49,301.89 - 321.14,304.33,207.26 - 354.86,"2,631,513","790,041",5.926B,--,81.17,3.95,"Dec 5, 2024 - Dec 9, 2024",--,--,290.13
2,ASTS,https://finance.yahoo.com/quote/ASTS/,30.61,+3.82,+14.25%,26.79,26.50 - 30.93,26.81,1.97 - 39.08,"14,861,596","16,209,161",8.446B,1.63,--,-1.30,"Nov 12, 2024 - Nov 18, 2024",--,--,37.98
3,BNTX,https://finance.yahoo.com/quote/BNTX/,105.00,+5.73,+5.77%,99.27,95.48 - 105.07,97.00,76.53 - 114.69,"1,813,273","736,598",24.965B,0.23,--,-2.34,"Nov 4, 2024",--,"Jun 2, 2022",110.97
4,ORCL,https://finance.yahoo.com/quote/ORCL/,163.02,+1.64,+1.02%,161.38,161.75 - 173.93,173.70,99.26 - 173.93,"15,667,225","8,163,180",451.729B,1.00,42.12,3.87,"Dec 9, 2024 - Dec 13, 2024",1.60 (0.99%),"Oct 10, 2024",164.59
5,TSLA,https://finance.yahoo.com/quote/TSLA/,229.39,-0.42,-0.18%,229.81,226.32 - 232.66,227.95,138.80 - 278.98,"32,806,206","97,169,109",732.806B,2.29,64.43,3.56,"Oct 18, 2024 - Oct 28, 2024",--,--,210.87
6,NVDA,https://finance.yahoo.com/quote/NVDA/,119.10,-0.04,-0.04%,119.14,117.60 - 119.96,119.01,39.23 - 140.76,"124,503,482","344,014,791",2.921T,1.67,55.91,2.13,"Nov 19, 2024 - Nov 25, 2024",0.04 (0.03%),"Sep 12, 2024",145.22
7,UBER,https://finance.yahoo.com/quote/UBER/,71.59,+3.50,+5.14%,68.09,70.94 - 72.75,72.40,40.09 - 82.14,"17,162,680","15,273,667",150.406B,1.33,77.78,0.92,"Nov 5, 2024 - Nov 11, 2024",--,--,87.02
8,BA,https://finance.yahoo.com/quote/BA/,161.99,-0.78,-0.48%,162.77,158.21 - 162.89,159.78,156.10 - 267.54,"5,578,877","6,261,159",99.822B,1.58,--,-5.65,"Oct 23, 2024 - Oct 28, 2024",--,"Feb 13, 2020",216.16
9,PLTR,https://finance.yahoo.com/quote/PLTR/,34.91,+0.06,+0.17%,34.85,34.53 - 35.40,34.86,13.68 - 35.40,"54,134,148","49,621,927",78.179B,2.71,205.35,0.17,"Oct 31, 2024 - Nov 4, 2024",--,--,26.94
